# IndicTrans2 HF Inference

We provide an example notebook on how to use our IndicTrans2 models which were originally trained with the fairseq to HuggingFace transformers for inference purpose.


## Setup

Please run the cells below to install the necessary dependencies.


In [1]:
%%capture
!git clone https://github.com/AI4Bharat/IndicTrans2.git

In [2]:
%%capture
%cd /content/IndicTrans2/huggingface_interface

In [3]:
%%capture
!python3 -m pip install nltk sacremoses pandas regex mock transformers>=4.33.2 mosestokenizer
!python3 -c "import nltk; nltk.download('punkt')"
!python3 -m pip install bitsandbytes scipy accelerate datasets
!python3 -m pip install sentencepiece

!git clone https://github.com/VarunGumma/IndicTransToolkit.git
%cd IndicTransToolkit
!python3 -m pip install --editable ./
%cd ..

**IMPORTANT : Restart your run-time first and then run the cells below.**

## Inference


In [1]:
import torch
from transformers import AutoModelForSeq2SeqLM, BitsAndBytesConfig, AutoTokenizer
from IndicTransToolkit import IndicProcessor

BATCH_SIZE = 4
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
quantization = None

In [2]:
def initialize_model_and_tokenizer(ckpt_dir, quantization):
    if quantization == "4-bit":
        qconfig = BitsAndBytesConfig(
            load_in_4bit=True,
            bnb_4bit_use_double_quant=True,
            bnb_4bit_compute_dtype=torch.bfloat16,
        )
    elif quantization == "8-bit":
        qconfig = BitsAndBytesConfig(
            load_in_8bit=True,
            bnb_8bit_use_double_quant=True,
            bnb_8bit_compute_dtype=torch.bfloat16,
        )
    else:
        qconfig = None

    tokenizer = AutoTokenizer.from_pretrained(ckpt_dir, trust_remote_code=True)
    model = AutoModelForSeq2SeqLM.from_pretrained(
        ckpt_dir,
        trust_remote_code=True,
        low_cpu_mem_usage=True,
        quantization_config=qconfig,
    )

    if qconfig == None:
        model = model.to(DEVICE)
        if DEVICE == "cuda":
            model.half()

    model.eval()

    return tokenizer, model


def batch_translate(input_sentences, src_lang, tgt_lang, model, tokenizer, ip):
    translations = []
    for i in range(0, len(input_sentences), BATCH_SIZE):
        batch = input_sentences[i : i + BATCH_SIZE]

        # Preprocess the batch and extract entity mappings
        batch = ip.preprocess_batch(batch, src_lang=src_lang, tgt_lang=tgt_lang)

        # Tokenize the batch and generate input encodings
        inputs = tokenizer(
            batch,
            truncation=True,
            padding="longest",
            return_tensors="pt",
            return_attention_mask=True,
        ).to(DEVICE)

        # Generate translations using the model
        with torch.no_grad():
            generated_tokens = model.generate(
                **inputs,
                use_cache=True,
                min_length=0,
                max_length=256,
                num_beams=5,
                num_return_sequences=1,
            )

        # Decode the generated tokens into text

        with tokenizer.as_target_tokenizer():
            generated_tokens = tokenizer.batch_decode(
                generated_tokens.detach().cpu().tolist(),
                skip_special_tokens=True,
                clean_up_tokenization_spaces=True,
            )

        # Postprocess the translations, including entity replacement
        translations += ip.postprocess_batch(generated_tokens, lang=tgt_lang)

        del inputs
        torch.cuda.empty_cache()

    return translations

Indic To English


In [5]:
#This block will take a lot of time to run.SETUP block.
indic_en_ckpt_dir = "ai4bharat/indictrans2-indic-en-1B"  # ai4bharat/indictrans2-indic-en-dist-200M
indic_en_tokenizer, indic_en_model = initialize_model_and_tokenizer(indic_en_ckpt_dir, quantization)
!pip install -q groq



import os

from groq import Groq

client = Groq(
    #api_key=userdata.get('GROQ_API_KEY'),
    api_key="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",
)
en_indic_ckpt_dir = "ai4bharat/indictrans2-en-indic-1B"  # ai4bharat/indictrans2-en-indic-dist-200M
en_indic_tokenizer, en_indic_model = initialize_model_and_tokenizer(en_indic_ckpt_dir, quantization)



In [6]:
# prompt: def translate_and_chat(input_text):
#     # First translation from Marathi to English
#     hi_sents = [input_text]
#     src_lang, tgt_lang = "mar_Deva", "eng_Latn"
#     en_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_en_model, indic_en_tokenizer, ip)
#     # Chat model interaction
#     chat_completion = client.chat.completions.create(
#         messages=[
#             {
#                 "role": "user",
#                 "content": en_translations[0],
#             }
#         ],
#         model="llama3-70b-8192",
#     )
#     # Getting the chat model's output
#     chat_output = chat_completion.choices[0].message.content
#     # Second translation from English back to Marathi
#     en_sents = [chat_output]
#     src_lang, tgt_lang = "eng_Latn", "mar_Deva"
#     hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)
#     # Return the final output
#     return hi_translations[0] use this as it is and print its output here



# Assuming necessary variables (indic_en_model, indic_en_tokenizer, etc.) are defined as in the provided code

# Initialize IndicProcessor

def translate_and_chat(input_text):
    # First translation from Marathi to English
    ip = IndicProcessor()
    hi_sents = [input_text]
    src_lang, tgt_lang = "mar_Deva", "eng_Latn"
    en_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_en_model, indic_en_tokenizer, ip)
    # Chat model interaction
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": en_translations[0],
            }
        ],
        model="llama3-70b-8192",
    )
    # Getting the chat model's output
    chat_output = chat_completion.choices[0].message.content
    # Second translation from English back to Marathi
    en_sents = [chat_output]
    src_lang, tgt_lang = "eng_Latn", "mar_Deva"
    hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)
    # Return the final output
    return hi_translations[0]

# Example usage:
# Assuming 'input_text' is defined somewhere
input_text = "नमस्कार, कसे आहात?" # Example Marathi text
output = translate_and_chat(input_text)
output

'मी फक्त एक भाषेचा आदर्श आहे, मला भावना किंवा भावना नाहीत, परंतु मी योग्यरित्या काम करत आहे आणि तुम्हाला कोणतेही प्रश्न किंवा कामे करण्यास मदत करण्यास तयार आहे. आज मी तुम्हाला कशी मदत करू शकतो? '

### English to Indic Example


In [7]:
!pip install gradio --upgrade

import gradio as gr
from gradio import Textbox, Interface



def translate_and_chat(input_text):
    ip = IndicProcessor()
    # First translation from Marathi to English
    hi_sents = [input_text]
    src_lang, tgt_lang = "mar_Deva", "eng_Latn"
    en_translations = batch_translate(hi_sents, src_lang, tgt_lang, indic_en_model, indic_en_tokenizer, ip)

    # Chat model interaction
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": en_translations[0],
            }
        ],
        model="llama3-70b-8192",
    )

    # Getting the chat model's output
    chat_output = chat_completion.choices[0].message.content

    # Second translation from English back to Marathi
    en_sents = [chat_output]
    src_lang, tgt_lang = "eng_Latn", "mar_Deva"
    hi_translations = batch_translate(en_sents, src_lang, tgt_lang, en_indic_model, en_indic_tokenizer, ip)

    # Return the final output
    return hi_translations[0]

# Define the Gradio interface
interface = Interface(
    fn=translate_and_chat,
    inputs=Textbox(lines=2, placeholder="Type your text here...", label="Enter Text in Marathi"),
    outputs=Textbox(label="Response in Marathi"),
    title="Marathi Chatbot",
    description="Enter a sentence in Marathi to chat with the bot.",
    article="<p style='text-align: center;'><img src='your_placeholder_image_url' alt='Placeholder Image' width='300'/></p>",
)


# Launch the interface
interface.launch()




   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 MB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.8 MB/s eta 0:00:00
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://965a57ecf590143b9c.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spac

In [8]:
!pip install gradio --upgrade

import gradio as gr
from gradio import Textbox, Dropdown

# Flores codes to language names mapping
LANGUAGE_NAMES = {
    "asm_Beng": "Assamese",
    "awa_Deva": "Awadhi",
    "ben_Beng": "Bengali",
    "bho_Deva": "Bhojpuri",
    "brx_Deva": "Bodo",
    "doi_Deva": "Dogri",
    "eng_Latn": "English",
    "gom_Deva": "Konkani",
    "guj_Gujr": "Gujarati",
    "hin_Deva": "Hindi",
    "hne_Deva": "Chhattisgarhi",
    "kan_Knda": "Kannada",
    "kas_Arab": "Kashmiri (Arabic)",
    "kas_Deva": "Kashmiri (Devanagari)",
    "kha_Latn": "Khasi",
    "lus_Latn": "Mizo",
    "mag_Deva": "Magahi",
    "mai_Deva": "Maithili",
    "mal_Mlym": "Malayalam",
    "mar_Deva": "Marathi",
    "mni_Beng": "Manipuri (Bengali)",
    "mni_Mtei": "Manipuri (Meitei)",
    "npi_Deva": "Nepali",
    "ory_Orya": "Odia",
    "pan_Guru": "Punjabi",
    "san_Deva": "Sanskrit",
    "sat_Olck": "Santali",
    "snd_Arab": "Sindhi (Arabic)",
    "snd_Deva": "Sindhi (Devanagari)",
    "tam_Taml": "Tamil",
    "tel_Telu": "Telugu",
    "urd_Arab": "Urdu"
}

def translate_and_chat(input_text, input_language):
    ip = IndicProcessor(inference=True)

    # Get language code from the selected language name
    lang_code = [code for code, name in LANGUAGE_NAMES.items() if name == input_language][0]

    # First translation from selected language to English
    input_sents = [input_text]
    en_translations = batch_translate(input_sents, lang_code, "eng_Latn", indic_en_model, indic_en_tokenizer, ip)

    # Chat model interaction
    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": en_translations[0],
            }
        ],
        model="llama3-70b-8192",
    )

    # Getting the chat model's output
    chat_output = chat_completion.choices[0].message.content

    # Second translation from English back to original language
    en_sents = [chat_output]
    hi_translations = batch_translate(en_sents, "eng_Latn", lang_code, en_indic_model, en_indic_tokenizer, ip)

    # Return the final output
    return hi_translations[0]

# Sort languages alphabetically for the dropdown
sorted_languages = sorted(LANGUAGE_NAMES.values())

# Define the Gradio interface with language selector
interface = gr.Interface(
    fn=translate_and_chat,
    inputs=[
        Textbox(lines=2, placeholder="Type your text here...", label="Input Text"),
        Dropdown(
            choices=sorted_languages,
            value="Marathi",
            label="Select Input Language"
        )
    ],
    outputs=Textbox(label="Response"),
    title="Multilingual Indian Language Chatbot",
    description="Select your language and enter text to chat with the bot.",
    examples=[
        ["पंतप्रधान नरेंद्र मोदींबद्दल तुम्हाला काय वाटतं?", "Marathi"],
        ["प्रधानमंत्री नरेंद्र मोदी के बारे में आप क्या सोचते हैं?", "Hindi"],
        ["What do you think about technology in India?", "English"],
        ["ভারতের প্রযুক্তি সম্পর্কে আপনি কী মনে করেন?", "Bengali"]
    ],
    theme="soft",
    allow_flagging="never"
)

# Launch the interface
interface.launch(share=True)

/usr/local/lib/python3.11/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated.Use `flagging_mode` instead.
  warnings.warn(


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d2b47f18a3541d0106.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
